<a href="https://colab.research.google.com/github/cysorianoc/IBM_Data_Science/blob/main/Capstone_P7_dash.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install jupyter-dash -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 38.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 27.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 228.0/228.0 kB 8.5 MB/s eta 0:00:00


In [ ]:
!pip3 install wget

  Preparing metadata (setup.py) ... done
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9656 sha256=d72d132bda578a2dfaf0d657ceaaf4c3e107ccec694e4a6b98e1c008ea6f3cf2
  Stored in directory: /root/.cache/pip/wheels/40/b3/0f/a40dbd1c6861731779f62cc4babcb234387e11d697df70ee97
Successfully built wget


In [ ]:
import wget

In [ ]:
data = wget.download( "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv")

### Explanation of the Code:

This Python script builds an **interactive web dashboard** using **Dash** to visualize SpaceX launch records. Below is a breakdown of the main components:

1. **Importing Required Libraries:**
   - `dash`: Used for building web applications.
   - `plotly.express`: For creating visualizations.
   - `pandas`: For handling data.

2. **Loading the Data:**
   - The script reads a CSV file (`spacex_launch_dash.csv`) containing SpaceX launch data.

3. **Creating the Dash Application:**
   - A `Dash` object is instantiated.

4. **Defining the App Layout:**
   - The layout includes:
     - A **title** (`html.H1`).
     - A **dropdown menu** (`dcc.Dropdown`) for selecting launch sites.
     - A **pie chart** (`dcc.Graph`) for displaying launch success statistics.
     - A **slider** (`dcc.RangeSlider`) for selecting payload mass range.
     - A **scatter plot** (`dcc.Graph`) to visualize the correlation between payload mass and launch success.

5. **Callbacks for Interactivity:**
   - **Pie Chart Update:**
     - If 'ALL' sites are selected, the pie chart displays the total count of successful and failed launches for all sites.
     - If a specific site is selected, the pie chart updates to show the success vs. failure count for that site.
   - **Scatter Chart Update:**
     - The scatter plot filters data based on the selected payload range.
     - If a specific site is selected, it filters by that site as well.

6. **Running the Application:**
   - `app.run_server(debug=True)` starts the web application.

### Summary:
- This script creates a **dynamic dashboard** where users can interact with the data using dropdowns and sliders.
- The **Pie Chart** shows launch success rates per site.
- The **Scatter Plot** visualizes the relationship between payload mass and launch success.
- The **callbacks ensure that charts update dynamically** based on user input.

In [ ]:
import dash  # Import the Dash library for building web applications
from dash import dcc, html  # Import components for UI elements
import plotly.express as px  # Import Plotly for data visualization
import pandas as pd  # Import pandas for handling data

# Load the dataset containing SpaceX launch records
# Ensure the file 'spacex_launch_dash.csv' is in the same directory as this script
# The dataset includes information on launch success, payload, and launch site

df = pd.read_csv('spacex_launch_dash.csv')

# Create a Dash web application instance
app = dash.Dash(__name__)

# Define the layout of the web application
app.layout = html.Div(children=[
    # Title of the Dashboard
    html.H1('SpaceX Launch Records Dashboard',
            style={'textAlign': 'center', 'color': '#503D36', 'font-size': 40}),

    # Dropdown menu to select a launch site
    dcc.Dropdown(
        id='site-dropdown',  # Unique identifier for this dropdown
        options=[
            {'label': 'All Sites', 'value': 'ALL'},
            {'label': 'CCAFS LC-40', 'value': 'CCAFS LC-40'},
            {'label': 'CCAFS SLC-40', 'value': 'CCAFS SLC-40'},
            {'label': 'KSC LC-39A', 'value': 'KSC LC-39A'},
            {'label': 'VAFB SLC-4E', 'value': 'VAFB SLC-4E'}
        ],
        value='ALL',  # Default selection
        placeholder="Select a Launch Site",
        style={'width': '50%', 'padding': '3px', 'font-size': '20px', 'text-align': 'center'}
    ),
    html.Br(),  # Line break for better spacing

    # Pie chart to display success rate
    html.Div(dcc.Graph(id='success-pie-chart')),
    html.Br(),

    # Label for the payload slider
    html.P("Payload range (Kg):"),

    # Slider to filter data based on payload mass
    dcc.RangeSlider(
        id='payload-slider',  # Unique identifier for this slider
        min=df['Payload Mass (kg)'].min(),
        max=df['Payload Mass (kg)'].max(),
        step=1000,  # Step size of the slider
        marks={i: str(i) for i in range(int(df['Payload Mass (kg)'].min()), int(df['Payload Mass (kg)'].max()), 1000)},
        value=[df['Payload Mass (kg)'].min(), df['Payload Mass (kg)'].max()]  # Default range
    ),

    # Scatter plot to show correlation between payload and launch success
    html.Div(dcc.Graph(id='success-payload-scatter-chart')),
])

# Callback function to update the pie chart when a launch site is selected
@app.callback(
    dash.dependencies.Output('success-pie-chart', 'figure'),  # Output is a figure for the pie chart
    dash.dependencies.Input('site-dropdown', 'value')  # Input is the selected launch site
)
def update_pie_chart(selected_site):
    if selected_site == 'ALL':
        # Group data by launch site and count success/failure
        success_counts = df.groupby(['Launch Site', 'class']).size().reset_index(name='count')
        fig = px.pie(
            success_counts,
            names='Launch Site',
            values='count',
            title="Total Launch Success vs. Failed for All Sites",
            color_discrete_sequence=px.colors.qualitative.Set3
        )
    else:
        # Filter data for the selected launch site
        filtered_df = df[df['Launch Site'] == selected_site]
        success_counts = filtered_df['class'].value_counts()
        fig = px.pie(
            names=success_counts.index,
            values=success_counts.values,
            title=f"Launch Success vs. Failed for {selected_site}"
        )
    return fig

# Callback function to update the scatter plot based on site selection and payload range
@app.callback(
    dash.dependencies.Output('success-payload-scatter-chart', 'figure'),  # Output is a figure for the scatter chart
    [
        dash.dependencies.Input('site-dropdown', 'value'),  # Input is the selected launch site
        dash.dependencies.Input('payload-slider', 'value')  # Input is the selected payload range
    ]
)
def update_scatter_chart(selected_site, payload_range):
    min_payload, max_payload = payload_range

    # Filter the data based on payload range
    filtered_df = df[(df['Payload Mass (kg)'] >= min_payload) & (df['Payload Mass (kg)'] <= max_payload)]

    if selected_site != 'ALL':
        # Further filter by the selected site if a specific site is chosen
        filtered_df = filtered_df[filtered_df['Launch Site'] == selected_site]

    # Create scatter plot showing correlation between Payload Mass and Launch Success
    fig = px.scatter(
        filtered_df,
        x='Payload Mass (kg)',
        y='class',
        color='Launch Site' if selected_site == 'ALL' else None,  # Color points by site if showing all sites
        labels={'Payload Mass (kg)': 'Payload Mass (kg)', 'class': 'Launch Success'},
        title=f"Payload vs. Launch Success for {selected_site}" if selected_site != 'ALL' else "Payload vs. Launch Success for All Sites"
    )
    return fig

# Run the Dash web application
if __name__ == '__main__':
    app.run_server(debug=True)  # Runs the app in debug mode for easier troubleshooting


<IPython.core.display.Javascript object>